![example](images/director_shot.jpeg)

# Recommendations for a New Movie Studio

**Authors:** Colin Pelzer, Daniel Burdeno, Emiko Naomasa, Piotr Czolpik
***

## Overview

Microsoft wants to create a new original video content creation studio. We utilized data from IMDb and The Numbers; such as, Total Gross, ROI, Run Time, Genres, and Directors. Data was researched, using the Python library Pandas, to perform exploratory analysis into what attributes are currently dominating the film industry, and in order to provide Microsoft reccomendations we used basic data visualization libraries like Matplotlib, and Seaborn.


## Business Problem

Microsoft has expressed interest in going into the original video creation by creating a new movie studio. 
The Studio team has tasked us to explore which types of films are dominating the market, 
and translate them into actionable intel that can help the head of Microsofts new studio make
an informed decision on which films they would like to start to produce.






## Data Understanding

Datasets were scraped from IMDb.com and The-Numbers.com, data from IMDb contains data regarding movie runtime, genre, and directors. The-Numbers dataset contains information relating to worldwide gross and production budget, which allowed us to calculate profit and return on investment(ROI). The dataset also included release dates, which allowed us to calculate the importance of season. We cleaned datasets to only contain information related the last ten years, to insure our reccomendations reflect the current market. Information from both datasets were joined to compare variables against total profit and ROI in order to produce meaningful recommendations.

In [141]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [142]:
df_crew = pd.read_csv('Data/zippedData/imdb.title.crew.csv.gz')
df_name = pd.read_csv('Data/zippedData/imdb.name.basics.csv.gz')
df_title_basic = pd.read_csv('Data/zippedData/imdb.title.basics.csv.gz')
df_num = pd.read_csv('Data/zippedData/tn.movie_budgets.csv.gz')

In [143]:
df_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  140417 non-null  object
 2   writers    110261 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [144]:
df_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


In [145]:
df_title_basic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [146]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


### Film Crew Information

This dataset contains the unique id number for the crew involved with making the corresponding movies, we used this dataset to obtain the names of film directors.

In [147]:
df_crew.head()

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [148]:
df_crew['directors'].value_counts()

nm3266654    62
nm2682776    48
nm5592581    48
nm3583561    46
nm0183659    44
             ..
nm4007484     1
nm9302300     1
nm3282701     1
nm4018983     1
nm5147791     1
Name: directors, Length: 98525, dtype: int64

Based on the intial look at the data, directors are seperated by a unique id name that we can cross reference with another database to pull their names, and the movies they worked on. We want to pull only movies with a single director, as they represent a majority of the data.

### Director Name Information

Using the information in this dataset, we can extract the name of the director from their unique id associated with them. We can now tie director name to the corresponding movie title's using both the previous and the next dataset (using tconst).

In [149]:
df_name.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


### Title Basics

This is one of the main datasets that we are going to use to create suggestions, useful information like title, genre, and runtime are all available to us within

In [150]:
df_title_basic.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [151]:
df_title_basic['start_year'].value_counts()
# Most of the information is from the past 10 years, however there are some funky data points so we will still hard code a limit into the data.

2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2012    13787
2011    12900
2010    11849
2019     8379
2020      937
2021       83
2022       32
2023        5
2024        2
2027        1
2026        1
2025        1
2115        1
Name: start_year, dtype: int64

In [152]:
df_title_basic['genres'].value_counts()
#Need to deal with multi-genre movies, and to trim down to the most relevant genres

Documentary                    32185
Drama                          21486
Comedy                          9177
Horror                          4372
Comedy,Drama                    3519
                               ...  
Animation,Crime                    1
Crime,Romance,Western              1
Family,Horror                      1
Documentary,Mystery,War            1
Animation,Biography,Fantasy        1
Name: genres, Length: 1085, dtype: int64

 Using the 'tconst' column, we can tie in the other supplementary datasets to include in our analysis. We can start to form questions off of this dataset such as, 'How does runtime affect the rating/ROI' or 'do certain genres dominate the market more than others?'

### The Numbers File

Three of the most important datapoints come from this set, production budget, worldwide gross, and release date. Utlizing production budget and worldwide gross we can compute both net profit and return on investment (ROI). This dataset will be on main set that we merge others into. We will need to trim it down to only contain movies from 2010 and onwards (last 10 years).

In [153]:
df_num.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [154]:
df_num['worldwide_gross'].describe()
#All the number data are objects, we will need to convert them to int/float to provide more analysis

count     5782
unique    5356
top         $0
freq       367
Name: worldwide_gross, dtype: object

Its a common idea that movies that come out during the summer and beginning of the holiday season perform better, using this data we can figure out how much it truly affects the total profitability.

## Data Preparation

Initial data preparation ranged between datasets. This included the removal of null values/incomplete entries, the transforming of certain data types into usable fields, and the trimming of data to only contain information from 2010-2021. It also included the creation of several new variables (profit, ROI) as well as the dropping of variables not used. This initial preparation and cleaning was done in order for us to merge all datasets into a final usable data frame from which we can draw meaniful conclusions/recommendations and produce our visuals.


### Film Crew Cleaning

In [155]:
# Delete 'writers' 
df_crew.drop(['writers'], axis=1, inplace=True)

# Drop missing values in 'directors'
df_crew.dropna(axis=0, subset=['directors'], inplace=True)

# Drop duplicate in 'tconst', film id
df_crew.drop_duplicates(subset='tconst', inplace=True)

# Split directors into director1, director2, director3, director4 
# 'director2' has 2nd director and others (if more than 2)
df_crew[['director1', 'director2']] = df_crew['directors'].str.split(',', n=1,  expand =True)

# Drop 'directors' and 'director2' columns.
# Only keeping director1 for each movie
df_crew.drop(['directors', 'director2'], axis=1, inplace=True)

In [156]:
df_crew.head()

,tconst,director1
0,tt0285252,nm0899854
2,tt0462036,nm1940585
3,tt0835418,nm0151540
4,tt0878654,nm0089502
5,tt0879859,nm2416460


### Director Name Cleaning

In [157]:
# Keep only columns 'nconst', 'primary_name'
df_name = df_name[['nconst', 'primary_name' ]]

# Checking for any null values
df_name.isna().value_counts()

nconst  primary_name
False   False           606648
dtype: int64

In [158]:
# Checking for any duplicate entries
df_name.duplicated().value_counts()

False    606648
dtype: int64

In [159]:
df_name.head()

,nconst,primary_name
0,nm0061671,Mary Ellen Bauder
1,nm0061865,Joseph Bauer
2,nm0062070,Bruce Baum
3,nm0062195,Axel Baumann
4,nm0062798,Pete Baxter


### Title Basics and Genre Cleaning

In [160]:
# Dropping all null values from dataset, incomplete entries would create merge conflicts
df_title_basic.dropna(inplace = True)

# Subset data to only include year 2010-2021
df_title_2010 = df_title_basic[(df_title_basic["start_year"] >= 2010) & (df_title_basic["start_year"] <= 2021)] 

# Drop the primary_title column, we will use original_title to avoid as many potential movies with the same name
df_title_final = df_title_2010.drop("primary_title", axis=1)

# Checking for any duplicate entries
df_title_final.duplicated().value_counts()

False    112229
dtype: int64

In [161]:
df_title_final.head()

,tconst,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama
4,tt0100275,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,2018,75.0,Comedy


### The Numbers Cleaning

In [162]:
# Define function to transform money values (object) into usable numbers (float)
def data_cleaning_money(column_name):
    df_num[column_name] = df_num[column_name].str.replace('$', '').str.replace(',', '').astype(float)

# Tranform product_budget and worldwide_gross and drop domestic_gross and id columns
data_cleaning_money('worldwide_gross')
data_cleaning_money('production_budget')
df_num = df_num.drop(columns = 'id')

# Transforming release_date into datetime object for easier use
df_num['release_date'] = pd.to_datetime(df_num['release_date'])

# Cleaning data and removing any potential null (0.0) values from relevant columns. 
df_num = df_num[df_num['domestic_gross'] != 0.0]
df_num = df_num[df_num['worldwide_gross'] != 0.0]
df_num = df_num[df_num['production_budget'] != 0.0]

# Drop any rows where domestic_gross = worldwide_gross, indictating no value for foreign gross
df_num = df_num[df_num['domestic_gross'] != df_num['worldwide_gross']]

# Drop domestic gross once cleaning is done
df_num_cleaned = df_num.drop('domestic_gross', axis = 1)

In [165]:
# Create new variable 'profit'
df_num_cleaned['profit'] = (df_num_cleaned['worldwide_gross'] - df_num_cleaned['production_budget'])

# Using 'profit' variable, create new variable "ROI" (Return On Investment)
df_num_cleaned['ROI'] = (df_num_cleaned['profit']) / (df_num_cleaned['production_budget'])



## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [163]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***